In [1]:
#]add Yao, BitBasis, Plots

In [2]:
using Yao, BitBasis

# Quantum Arithmetic with QFT

One way to perform arithmetic on a quantum computer is to convert the classical circuit to its reversible quantum equivalent[<sup>1</sup>](https://youtu.be/TL8ZI_goX04) [<sup>2</sup>](https://quantumcomputinguk.org/tutorials/performing-addition-on-ibms-quantum-computers). However, the ideal algorithm for a quantum computer may not be similar to its classical counterpart.

For example, Tomas Draper [proposed](http://arxiv.org/abs/quant-ph/0008033) an addition circuit using the [quantum fourier transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform) (QFT). This was later [extended](http://arxiv.org/abs/1411.5949) to include weighted sums and multiplication.

This notebook implements the algorithms in [Yao](https://yaoquantum.org/).

Throughout these algorithms heavy use of a particularly useful phase-shift operator is used.

$$R_l = \begin{pmatrix} 1 & 0 \\ 0 & e ^ \frac{ 2 \pi i }{2 ^ l}  \end{pmatrix}$$

Very roughly speaking it's used to transform a binary number encoded as basis states $a = a_n 2^{n-1} + a_{n-1} 2^{n-2} + ... + a_2 2^1  + a_1 2^0$  into a phase encoding. We'll define that operator first along with its controlled version.

In [3]:
R(l) = shift(2π/(2^l))
CR(i, j, l) = control(i, j=>R(l))

@assert mat(R(0)) ≈ mat(I2)

The algorithm itself takes two values $a$ and $b$, computes $QFT(a)$ and then uses $b$ to evolve $QFT(a)$ into $QFT(a + b)$. The inverse transform $QFT^{-1}(a+b)$ may then be applied for the final result. 

To begin we implement the QFT from scratch exactly as described in the paper. This uses the controlled phase gate defined above multiple times.

<div>
<img src="quantum-arithmetic-qft.png" width="600"/>
</div>

It's important to note that the order of the bits arbitarily starts with the least significant bit at the bottom. This convention is used throughout the paper and this notebook.

In [4]:
A(n, k) = chain(n, j==1 ? put(k=>H) : CR(k-j+1, k, j) for j in 1:k)

qft(n) = chain(A(n, k) for k in n:-1:1)

qft(3)

nqubits: 3
chain
├─ chain
│  ├─ put on (3)
│  │  └─ H
│  ├─ control(2)
│  │  └─ (3,) shift(1.5707963267948966)
│  └─ control(1)
│     └─ (3,) shift(0.7853981633974483)
├─ chain
│  ├─ put on (2)
│  │  └─ H
│  └─ control(1)
│     └─ (2,) shift(1.5707963267948966)
└─ chain
   └─ put on (1)
      └─ H


The inverse is just the transpose, so we can check that.

In [5]:
@assert zero_state(4) |> qft(4) |> qft(4)' ≈ zero_state(4)

# Quantum Adder

The complete adder circuit consists of an initial preparatory block performing the QFT and a final block applying the inverse. Sandwiched between them is a second transformation which evolves the QFT. Again this circuit is taken exactly as described in the paper.

<p>
<img src="quantum-arithmetic-adder.png" width="600"/>
</p>

This assumes both numbers have the same number of bits, $n$. Since the sum is produced by transforming the $a$ register in place, the result will be $mod$ $2^n$. Extending the circuit to include a carry qubit is straightforward[<sup>3</sup>](https://deepai.org/publication/quantum-arithmetic-operations-based-on-quantum-fourier-transform-on-signed-integers).

In [6]:
B(n, k) = chain(n*2, CR(k+n-j+1, k, j) for j in 1:k)

add(n) = chain(B(n, k) for k in n:-1:1)

add(3)

nqubits: 6
chain
├─ chain
│  ├─ control(6)
│  │  └─ (3,) shift(3.141592653589793)
│  ├─ control(5)
│  │  └─ (3,) shift(1.5707963267948966)
│  └─ control(4)
│     └─ (3,) shift(0.7853981633974483)
├─ chain
│  ├─ control(5)
│  │  └─ (2,) shift(3.141592653589793)
│  └─ control(4)
│     └─ (2,) shift(1.5707963267948966)
└─ chain
   └─ control(4)
      └─ (1,) shift(3.141592653589793)


Putting al the pieces together we're ready to build the full circuit. 

<p>
<img src="quantum-arithmetic-adder-circuit.png" width="500"/>
</p>


In [7]:
function adder(a,b) 
  @assert length(b) == length(a)
  n = length(a)
  product_state(bcat(b,a)) |>
    focus!(1:n) |> qft(n) |>
    relax! |>
    add(n) |>
    focus!(1:n) |> qft(n)' |>
    measure! |> bint
end    

adder (generic function with 1 method)

The `focus!` and `relax!` fuctions are useful for applying the QFT to only $a$'s qubits.

Let's test it with a couple of numbers.

In [8]:
a = bit"100101110"
b = bit"010001101"

@assert (bint(a) + bint(b))%(2^length(a)) == adder(a,b)

In [9]:
@show bint(a), bint(b), adder(a,b);

(bint(a), bint(b), adder(a, b)) = (302, 141, 443)
